In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')

In [3]:
neighborhoods_table = soup.find('table',{"class":'wikitable sortable'})
for table in neighborhoods_table:
    rows = neighborhoods_table.find_all('tr')

The above cells get the table from the wikipedia link.

The below cell gets the 3 headers from the table.

In [4]:
headers = [th.text.rstrip() for th in rows[0].find_all('th')]

In [5]:
PostalCode = []
Borough = []
Neighborhood = []

In [6]:
str1 = 'Not assigned'
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        if str1 not in str(cells[1]):
            postal = cells[0]
            PostalCode.append(postal.text.strip())
            boro = cells[1]
            Borough.append(boro.text.strip())
            neigh = cells[2]
            Neighborhood.append(neigh.text.strip())

The above cells make arrays for the data in the table, removing the Not Assigned neighborhoods.

The below cells put the data into a dictionary, and from there into a dataframe.

In [7]:
data = {'Postal Code':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}

In [8]:
dataframe1 = pd.DataFrame.from_dict(data)

In [9]:
dataframe1.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [10]:
dataframe1.shape

(103, 3)

The below cells add the given coordinates CSV file, and then merges that dataframe with the existing dataframe on the Postal Code value.

In [11]:
coords = pd.read_csv("http://cocl.us/Geospatial_data")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
CombinedData = pd.merge(dataframe1,coords,on = 'Postal Code')
CombinedData.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


The below cell installs and imports Folium for mapping

In [13]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print("Imported folium")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  conda-forge
    ------------------------------------------------------------
                       

The below cells create the map of Toronto and add markers for the neighborhood.    

In [53]:
map_toronto = folium.Map(location=[43.711012, -79.391726], zoom_start=12) 

In [55]:
for lat, lng, borough, neighborhood in zip(CombinedData['Latitude'], CombinedData['Longitude'], CombinedData['Borough'], CombinedData['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)    

In [56]:
map_toronto